In [1]:
import pandas as pd 
import numpy as np
import os

In [2]:
ts_df = pd.read_csv("/Users/george/Desktop/LW-DDI Project/twosides.csv")

In [3]:
fe_df = pd.read_csv("/Users/george/Desktop/LW-DDI Project/cxe_feat_eng_droppedcolumns.csv")

In [4]:
def df_optimized(df, verbose=True, **kwargs):
    """
    Reduces size of dataframe by downcasting numerical columns
    :param df: input dataframe
    :param verbose: print size reduction if set to True
    :param kwargs:
    :return:
    """
    in_size = df.memory_usage(index=True).sum()
    for type in ["float", "integer"]:
        l_cols = list(df.select_dtypes(include=type))
        for col in l_cols:
            df[col] = pd.to_numeric(df[col], downcast=type)
            if type == "float":
                df[col] = pd.to_numeric(df[col], downcast="integer")
    out_size = df.memory_usage(index=True).sum()
    ratio = (1 - round(out_size / in_size, 2)) * 100
    GB = out_size / 1000000000
    if verbose:
        print("optimized size by {} % | {} GB".format(ratio, GB))
    return df

In [5]:
fe_df= df_optimized(fe_df)

optimized size by 65.99999999999999 % | 0.001197632 GB


In [6]:
ts_df

,Unnamed: 0,Drug1_ID,Drug1,Drug2_ID,Drug2,Y
0,0,CID000002173,CC1(C(N2C(S1)C(C2=O)NC(=O)C(C3=CC=CC=C3)N)C(=O...,CID000003345,CCC(=O)N(C1CCN(CC1)CCC2=CC=CC=C2)C3=CC=CC=C3,767
1,1,CID000002173,CC1(C(N2C(S1)C(C2=O)NC(=O)C(C3=CC=CC=C3)N)C(=O...,CID000003345,CCC(=O)N(C1CCN(CC1)CCC2=CC=CC=C2)C3=CC=CC=C3,25
2,2,CID000002173,CC1(C(N2C(S1)C(C2=O)NC(=O)C(C3=CC=CC=C3)N)C(=O...,CID000003345,CCC(=O)N(C1CCN(CC1)CCC2=CC=CC=C2)C3=CC=CC=C3,85
3,3,CID000002173,CC1(C(N2C(S1)C(C2=O)NC(=O)C(C3=CC=CC=C3)N)C(=O...,CID000003345,CCC(=O)N(C1CCN(CC1)CCC2=CC=CC=C2)C3=CC=CC=C3,735
4,4,CID000002173,CC1(C(N2C(S1)C(C2=O)NC(=O)C(C3=CC=CC=C3)N)C(=O...,CID000003345,CCC(=O)N(C1CCN(CC1)CCC2=CC=CC=C2)C3=CC=CC=C3,959
...,...,...,...,...,...,...
4649436,929883,CID000000453,C(C(C(C(C(CO)O)O)O)O)O,CID000004595,CC1=NC=CN1CC2CCC3=C(C2=O)C4=CC=CC=C4N3C,912
4649437,929884,CID000005391,CN1C2=C(C=C(C=C2)Cl)C(=NC(C1=O)O)C3=CC=CC=C3,CID000068740,C1=CN(C=N1)CC(O)(P(=O)(O)O)P(=O)(O)O,1281
4649438,929885,CID000002909,CCC1C(=O)N(CC(=O)N(C(C(=O)NC(C(=O)N(C(C(=O)NC(...,CID000005978,CCC1(CC2CC(C3=C(CCN(C2)C1)C4=CC=CC=C4N3)(C5=C(...,289
4649439,929886,CID000003937,C1CC(N(C1)C(=O)C(CCCCN)NC(CCC2=CC=CC=C2)C(=O)O...,CID000077999,CN(CCOC1=CC=C(C=C1)CC2C(=O)NC(=O)S2)C3=CC=CC=N3,2


In [7]:
drug1_df = ts_df.copy()

In [8]:
drug1_df = drug1_df[['Drug1']]

In [9]:
drug1_merged = drug1_df.merge(fe_df, on='Drug1', how='left')

In [10]:
drug2_df =ts_df[['Drug2','Y']]

In [11]:
f2_df_copy = fe_df.rename(columns ={'Drug1':'Drug2'})

In [12]:
drug2_merged = drug2_df.merge(f2_df_copy, on='Drug2', how='left')

In [13]:
final_merge = pd.concat([drug1_merged,drug2_merged],axis =1)

In [15]:
# final_merge.to_csv('/Users/george/Desktop/LW-DDI Project/final_merge2.csv')

In [21]:
final_merge # drop unnamed. 

0               CCC(=O)N(C1CCN(CC1)CCC2=CC=CC=C2)C3=CC=CC=C3
1               CCC(=O)N(C1CCN(CC1)CCC2=CC=CC=C2)C3=CC=CC=C3
2               CCC(=O)N(C1CCN(CC1)CCC2=CC=CC=C2)C3=CC=CC=C3
3               CCC(=O)N(C1CCN(CC1)CCC2=CC=CC=C2)C3=CC=CC=C3
4               CCC(=O)N(C1CCN(CC1)CCC2=CC=CC=C2)C3=CC=CC=C3
                                 ...                        
4649436              CC1=NC=CN1CC2CCC3=C(C2=O)C4=CC=CC=C4N3C
4649437                 C1=CN(C=N1)CC(O)(P(=O)(O)O)P(=O)(O)O
4649438    CCC1(CC2CC(C3=C(CCN(C2)C1)C4=CC=CC=C4N3)(C5=C(...
4649439      CN(CCOC1=CC=C(C=C1)CC2C(=O)NC(=O)S2)C3=CC=CC=N3
4649440    C(C1C(C(C(C(O1)OC2(C(C(C(O2)OS(=O)(=O)O)OS(=O)...
Name: Drug2, Length: 4649441, dtype: object